# Quantum Watermarking Greyscale Images

### Authors: Luuk Coopmans, Cillian Doherty, Maria Graham, Ian Jubb and Rajarshi Tiwari

In this Jupyter Notebook we show the code we wrote for the Qiskit Quantum Summer Jam. Our code implements Quantum Information Processing (QIP) with as application the watermarking and scrambling of quantum images in Qiskit. In this file we first define and explain the functions we need for QIP and quantum watermarking, then we show an example of watermarking a $16\times16$ greyscale image of a Schrödinger Cat with an IBM Logo. In the end we also show how to scramble the watermark such that it is protected from intruders.   

### Defining the required functions for QIP and watermarking

We start with importing the required modules and packages. For version requirements see the Readme in our Github.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
from qiskit import *

Next we define a little function that can convert color images to greyscale images in the range (0, 255). We needed this because we figured out that the method presented in [1] to encode colored images is not accurate enough when running a quantum simulator. 

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

To encode the classical carrier image and the watermark into a quantum state we need a function that converts the greyscale values of the pixels indexed by $k$ to qubit angles $\theta_{k}$ and then computes the coefficients of the state vector of the quantum state. Here we use the $\theta_{k}=2*\arccos(G^{k}/255)$ for the angles since this avoids precision problems (taking the $\arccos$ of arguments bigger than one) when retrieving them later back again from the measured probabilities of the quantum image. 

In [ ]:
def get_Coeff(Image, watermark=None):
    """ Function that takes in an Greyscale Image matrix of any size and and optional
        watermark image and shape and returns qubit coefficients in the computational
        basis. 
        TO DO: build in a check that ensures the sizes of the Image and watermark are the same
    """
    
    Flatten_Image = np.reshape(Image, np.shape(Image)[0]*np.shape(Image)[1])
    
    # Get the theta angles of the Image and watermark from the greyscale values
    Theta_I = 2*np.arccos(np.sqrt(Flatten_Image/255))
                          
    if watermark is None:
        Theta_W = np.zeros(len(Theta_I))
    else:  
        Flatten_watermark = np.reshape(watermark, np.shape(watermark)[0]*np.shape(watermark)[1])
        Theta_W = 2*np.arccos(np.sqrt(Flatten_watermark/255))
        
        
    # Compute the qubit Coeff and return them in a normalized qubit state vector. 
    Coeff = np.stack((np.cos(Theta_I/2)*np.cos(Theta_W/2),
                       np.cos(Theta_I/2)*np.sin(Theta_W/2),
                       np.sin(Theta_I/2)*np.cos(Theta_W/2),
                       np.sin(Theta_I/2)*np.sin(Theta_W/2)))
    
    
    Coeff = np.ndarray.flatten(np.transpose(Coeff))
    
    return Coeff/np.linalg.norm(Coeff)

We then wrote a function to initialize a general quantum state vector into a quantum circuit for qiskit we wrote the following fuction

In [ ]:
def initialise_state(desired_initial_state):
    """ Returns an initialized quantum circuit for a given input state"""
    n = int(round(np.log2(desired_initial_state.size))) #number of qubits
    
    qc_init = QuantumCircuit(n)

    qc_init.initialize(desired_initial_state, range(n))
    return qc_init

Finally to get the newly measured classical images back out from the measured probabilities of the quantum circuit we define the function get_Images. For this we combine the probabilities $Pr^{k}(00)$, $Pr^{k}(01)$, $Pr^{k}(10)$ and $Pr^{k}(11)$ for each binary qubit $k$ and the corresponding collapsed greyscale qubit states $0$ and $1$ for for both the image and watermark. Moreover we normalize these probabilities for each $k$ to have a higher precision.

In [ ]:
def get_Images(probability_vector):
    """ Takes in an probability vector obtained from measuring all the qubits in the quantum circuit and returns
        the corresponding matrices of the greyscale values of the measured images.
        TO DO: vectorize this function
    """
    
    n = int(np.round(len(probability_vector)/4)) # get sizes back of classical images
    c_Image = np.zeros(n)
    w_Image = np.zeros(n)
    
    # Sum and normalize the probabilities to get the greyscale pixel value out
    for i in range(n):
        c_Image[i] = (probability_vector[4*i]+probability_vector[4*i+1])*255/np.sum(probability_vector[4*i:(4*i)+4])
        w_Image[i] = (probability_vector[4*i]+probability_vector[4*i+2])*255/np.sum(probability_vector[4*i:(4*i)+4])
    
    c_Image = np.reshape(c_Image,[np.int(np.round(np.sqrt(n))),np.int(np.round(np.sqrt(n)))])
    w_Image = np.reshape(w_Image,[np.int(np.round(np.sqrt(n))),np.int(np.round(np.sqrt(n)))])
    
    return c_Image, w_Image

### Quantum Watermarking a Schrödinger Cat image with an IBM Logo

We first read in our classical images and convert them to greyscale. We also resize them to the wanted $16\times16$ shape and visualize them.

In [ ]:
N = 16 #size in pixels


# Carrier Image
img_file = os.path.expanduser("cat.png")
img = cv2.imread(img_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
res = cv2.resize(img, dsize=(N, N), interpolation=cv2.INTER_CUBIC)
C_Image = rgb2gray(res)

# Watermark
img_file2 = os.path.expanduser("IBM-logo.png")
img2 = cv2.imread(img_file2)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
res2 = cv2.resize(img2, dsize=(N, N), interpolation=cv2.INTER_CUBIC)
W_Image = rgb2gray(res2)

In [ ]:
plt.rcParams['figure.figsize'] = [9, 7]
plt.rcParams['figure.dpi'] = 100

fig, axs = plt.subplots(1,2)

axs[0].imshow(C_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0].title.set_text('Carrier Image')

axs[1].imshow(W_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[1].title.set_text('Watermark Image')

Next we compute and initialize quantum state vector for the watermarked quantum image. To show to initialized state in the quantum circuit one can uncomment the 3rd line for relatively small system sizes (number of qubits).

In [ ]:
State_vector = get_Coeff(C_Image, W_Image)
qc_init = initialise_state(State_vector)
#qc_init.draw('mpl') 

Subsequently we build the quantum circuit that initializes and measures the watermarked quantum image. We also define how many shots we want on the quantum simulator. Again there is an option to visualize the circuit.

In [ ]:
n = int(round(np.log2(State_vector.size))) # number of qubits
shots0 = 100000

# Create a Quantum Circuit
meas = QuantumCircuit(n, n)
meas.barrier(range(n))

# map the quantum measurement to the classical bits
meas.measure(range(n), range(n))

# The Qiskit circuit object supports composition using
# the addition operator.
qc = qc_init + meas

#drawing the circuit
#qc.draw('mpl')

We run the circuit using the Aer's qasm_simulator but in the next cell also provide optional commented code to run it on one of IBMs real devices and its quantum simulator. We note that the initialize function is not available on all the IBM machines. Also you need an account on IBMQ to run this on their machines. 

In [ ]:
# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
job_sim = execute(qc, backend_sim, shots=shots0)

# Grab the results from the job.
result_sim = job_sim.result()

In [ ]:
#from qiskit import IBMQ
#IBMQ.load_account()
#provider = IBMQ.get_provider("ibm-q")
#backend = provider.get_backend("ibmq_essex")
#job = q.execute(qc, backend=backend, shots=shots0)
#job_monitor(job)

After the job is completed we get out the number of counts of each state and convert them to probabilities.

In [ ]:
counts = result_sim.get_counts(qc)
probability_vector = np.zeros(2**n)
int_counts = counts.int_outcomes()

for k in range(2**n):
    probability_vector[k] = int_counts.get(k, 0)/shots0

Finally we get and plot our retrieved watermark and carrier image back from the get_Images function defined above.

In [ ]:
RC_Image, RW_Image = get_Images(probability_vector)

In [ ]:
fig, axs = plt.subplots(2,2)

axs[0,0].imshow(RC_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0,0].title.set_text('Retrieved Carrier Image')

axs[0,1].imshow(C_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0,1].title.set_text('Original Carrier Image')

axs[1,0].imshow(RW_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[1,0].title.set_text('Retrieved Watermark Image')

axs[1,1].imshow(W_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[1,1].title.set_text('Original Watermark Image')

plt.show()

### Scrambling the watermark image

To scramble the quantum image we apply a few controlled $U_3$ rotations to the greyscale qubit of the watermark controlled by some of the qubits encoded the pixel index $k$. We start again from the originally initialized watermarked quantum image qc_init.

In [ ]:
n = int(round(np.log2(State_vector.size)))
shots0 = 100000

# Create the Quantum Scramble Circuit with controlled U3 rotations 
meas = QuantumCircuit(n, n)
meas.barrier(range(n))
meas.cu3(np.pi/4, 0, 0, 3, 0)
meas.cu3(-np.pi/4, 0, 0, 4, 0) 
meas.cu3(-np.pi/2, 0, 0, 5, 0) 
meas.cu3(-np.pi/3, 0, 0, 6, 0) 
meas.cu3(np.pi/2, 0, 0, 7, 0) 

# map the quantum measurement to the classical bits
meas.measure(range(n), range(n))

# The Qiskit circuit object supports composition using
# the addition operator.
qc = qc_init + meas

# Optional uncomment to draw the circuit
#qc.draw('mpl')

We run the new circuit on the simulator.

In [ ]:
# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=shots0)

# Grab the results from the job.
result_sim = job_sim.result()

We can convert the measurement results again the images with the get_Images function.

In [ ]:
counts = result_sim.get_counts(qc)
probability_vector = np.zeros(2**n)
int_counts = counts.int_outcomes()

for k in range(2**n):
    probability_vector[k] = int_counts.get(k, 0)/shots0

In [ ]:
RRC_Image, SCrW_Image = get_Images(probability_vector)

In [ ]:
fig, axs = plt.subplots(2,2)

axs[0,0].imshow(RRC_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0,0].title.set_text('Retrieved Carrier Image')

axs[0,1].imshow(C_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0,1].title.set_text('Original Carrier Image')


axs[1,0].imshow(SCrW_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[1,0].title.set_text('Scrambled Watermark Image')

axs[1,1].imshow(W_Image/255, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[1,1].title.set_text('Original Watermark Image')

plt.show()

### Thanks for working through our code and feel free to contact us if there are any questions!